In [2]:
lakehouse_silver_table=spark.conf.get('spark.executorEnv.lakehouse_silverTable')
print(lakehouse_silver_table)

StatementMeta(, ced8c32c-4381-45f8-b484-6c2bafb53e45, 4, Finished, Available, Finished)

abfss://Fabric_E2E@onelake.dfs.fabric.microsoft.com/Lakehouse_Silver_.Lakehouse/Tables


In [6]:
from pyspark.sql.functions import expr, sequence, explode, to_date

# Define start and end date
start = "2011-01-01"
end = "2022-12-12"

# Create a sequence of dates and explode them into rows
date_df = spark.sql(f"""
  SELECT explode(sequence(to_date('{start}'), to_date('{end}'), interval 1 day)) AS Date
""")

date_df.printSchema()

# Enrich with calendar attributes
date_df_transformed = date_df \
    .withColumn("DIM_DateId", expr("date_format(Date, 'yyyyMMdd')"))\
    .withColumn('Date',to_date('Date','yyyy-MM-dd'))\
    .withColumn("Year", expr("year(Date)")) \
    .withColumn("Month", expr("month(Date)")) \
    .withColumn("Day", expr("day(Date)")) \
    .withColumn("Weekday", expr("date_format(Date, 'EEEE')"))

date_df_transformed.printSchema()

# Reorder columns
date_df_clean = date_df_transformed.select("DIM_DateId", "Date", "Year", "Month", "Day", "Weekday")


StatementMeta(, ced8c32c-4381-45f8-b484-6c2bafb53e45, 8, Finished, Available, Finished)

root
 |-- Date: date (nullable = false)

root
 |-- Date: date (nullable = true)
 |-- DIM_DateId: string (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Weekday: string (nullable = true)



In [7]:
# Save as Delta table
date_df_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{lakehouse_silver_table}/DIM_Date")

StatementMeta(, ced8c32c-4381-45f8-b484-6c2bafb53e45, 9, Finished, Available, Finished)